In [1]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication
import os

### Login to OpenShift Cluster

In [2]:
auth = TokenAuthentication(
    # Get from `oc whoami -t`
    token = "sha256~EQP2L9gKGuKf0RBIpySzeUCLrQzAC8fWdwIJRt9XPKU",
    # Get from `oc whoami --show-server`
    server = "https://api.cluster-7vbv2.7vbv2.sandbox2425.opentlc.com:6443",
    skip_tls=True
)
auth.login()

Insecure request warnings have been disabled


'Logged into https://api.cluster-7vbv2.7vbv2.sandbox2425.opentlc.com:6443'

### Create Ray Cluster

In [3]:
cluster = Cluster(ClusterConfiguration(
    name='raytest', 
    namespace='agentic-workshop', 
    worker_extended_resource_requests={"nvidia.com/gpu": 1},
    num_workers=3,
    worker_cpu_requests=2,
    worker_cpu_limits=2,
    worker_memory_requests=8,
    worker_memory_limits=8,
    # image="quay.io/rhoai/ray:2.35.0-py39-cu121-torch24-sh20",
    image="quay.io/rhoai/ray:2.35.0-py311-cu121-torch24-fa26",
))

Yaml resources loaded for raytest


Output()

In [4]:
# Bring up the Ray cluster
cluster.up()

Ray Cluster: 'raytest' has successfully been created


In [5]:
# Check cluster status
cluster.status()

                        🚀 CodeFlare Cluster Status 🚀                       
                                                                             
 ╭─────────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                                  │ 
 │   raytest                                                 Inactive ❌   │ 
 │                                                                         │ 
 │   URI: ray://raytest-head-svc.agentic-workshop.svc:10001                │ 
 │                                                                         │ 
 │   ]8;id=836879;None\Dashboard🔗]8;;\                                                           │ 
 │                                                                         │ 
 ╰─────────────────────────────────────────────────────────────────────────╯

(<CodeFlareClusterStatus.STARTING: 2>, False)

In [6]:
# Wait for Ray cluster to be ready
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [7]:
# Check cluster details
cluster.details()

                      🚀 CodeFlare Cluster Details 🚀                      
                                                                           
 ╭───────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                                │ 
 │   raytest                                                 Active ✅   │ 
 │                                                                       │ 
 │   URI: ray://raytest-head-svc.agentic-workshop.svc:10001              │ 
 │                                                                       │ 
 │   ]8;id=958466;https://ray-dashboard-raytest-agentic-workshop.apps.cluster-7vbv2.7vbv2.sandbox2425.opentlc.com\Dashboard🔗]8;;\                                                         │ 
 │                                                                       │ 
 │                       Cluster Resources                               │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮           │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │           │ 
 │   │             │  │                                      │           │ 
 │   │  3          │  │  8G~8G       2~2         1           │           │ 
 │   │             │  │                                      │           │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯           │ 
 ╰───────────────────────────────────────────────────────────────────────╯

RayCluster(name='raytest', status=<RayClusterStatus.READY: 'ready'>, head_cpu_requests=2, head_cpu_limits=2, head_mem_requests='8G', head_mem_limits='8G', num_workers=3, worker_mem_requests='8G', worker_mem_limits='8G', worker_cpu_requests=2, worker_cpu_limits=2, namespace='agentic-workshop', dashboard='https://ray-dashboard-raytest-agentic-workshop.apps.cluster-7vbv2.7vbv2.sandbox2425.opentlc.com', worker_extended_resources={'nvidia.com/gpu': 1}, head_extended_resources={})

### Ray Job Submission

In [8]:
client = cluster.job_client

In [9]:
submission_id = client.submit_job(
    entrypoint="python ray_fine_tuning.py",
    runtime_env={
        "env_vars": {
            'AWS_ACCESS_KEY_ID': 'minio',
            'AWS_SECRET_ACCESS_KEY': 'minio123',
            'AWS_DEFAULT_REGION': 'none',
            'AWS_S3_ENDPOINT': 'http://minio-service.minio.svc.cluster.local:9000'
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["*.ipynb", "*.md"]
    },
)
print(submission_id)

2025-08-04 01:13:34,187	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_704f29fff3b90f7c.zip.
2025-08-04 01:13:34,188	INFO packaging.py:530 -- Creating a file package for local directory './'.


raysubmit_ydH9K3x7Bcv1eEu7


### Clean up

In [10]:
# client.stop_job(submission_id)

In [11]:
# client.delete_job(submission_id)

In [12]:
# cluster.down()

#Install TensorBoard in the Ray head node

In [ ]:
# kubectl exec `kubectl get pod -l ray.io/node-type=head -o name` -- pip install tensorboard

Start TensorBoard server: 

In [ ]:
# kubectl exec `kubectl get pod -l ray.io/node-type=head -o name` -- tensorboard --logdir /tmp/ray --bind_all --port 6006

Port-foward the TensorBoard UI endpoint: 

In [ ]:
# kubectl port-forward `kubectl get pod -l ray.io/node-type=head -o name` 6006:6006
# also add 6007 port with ray-head-service
# raytest-head-svc
	
# app.kubernetes.io/created-by=kuberay-operator, app.kubernetes.io/name=kuberay, ray.io/cluster=raytest, ray.io/identifier=raytest-head, ray.io/node-type=head
	
# 172.30.71.76:10001
# 172.30.71.76:8265
# 172.30.71.76:6379
# 172.30.71.76:8080
# 172.30.71.76:6007
# oc expose svc/rayclient-raytest with 6007